In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt 

import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from consts import HERE

sys.path.append(str(HERE.parent))
import pandas as pd

In [ ]:
# plt.clf()
# plt.figure()
# idx = 90
# time = 10000
# flu = moving_average(dff[idx, :time] - np.min(dff[idx, :time]), 10)
# c = deconv[idx, :time]

# plt.plot(flu, color='blue', alpha=0.7)
# plt.plot(c, color='red')

In [ ]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
# rate_paths = list((HERE.parent / "cache"  ).glob("*-all-spike-rate.npy"))

# type = 'all-spike-rate.npy'
type = 'resting-rates.npy'

suite2p_type = "activity_suite2p_rates"

if suite2p_type != "activity_suite2p_rates":
    rate_paths = list((HERE.parent / "cache").glob(f"*-{type}"))
else:
    rate_paths = list((HERE.parent /  "cache" / suite2p_type).glob(f"*-{type}"))


print(rate_paths)

wt_color = sns.color_palette("muted")[0]
ctko_color = sns.color_palette("muted")[1]

data = {}
palette = {}

for path in rate_paths:
    mouse = path.name.split("-")[0]
    assert mouse[:3] == "J02"

    # Even numbers are WT
    genotype = "WT" if int(mouse[3]) % 2 == 0 else "CTKO"
    data[mouse] = {"genotype": genotype, "data": np.load(path)}
    palette[mouse] = wt_color if genotype == "WT" else ctko_color



df = pd.concat(
    [
        pd.DataFrame({"Subject": subject, "Genotype": info["genotype"], "Firing rate": info["data"]})
        for subject, info in data.items()
    ],
).reset_index(drop=True)


[PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J026-2024-10-24-resting-rates.npy'), PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J025-2024-09-27-resting-rates.npy'), PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J023-2024-09-27-resting-rates.npy'), PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J022-2024-09-27-resting-rates.npy'), PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J029-2024-10-25-resting-rates.npy'), PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J024-2024-10-09-resting-rates.npy'), PosixPath('/Users/jamesrowland/Code/ctko/cache/activity_suite2p_rates/J027-2024-10-09-resting-rates.npy')]


In [ ]:
plt.figure(figsize=(10, 6))
genotype_colors = {'WT': 'green', 'CTKO': 'orange'}
ax = sns.boxplot(x='Genotype', y='Firing rate', hue='Subject', data=df, dodge=True, showfliers=False, legend=True)
plt.ylabel("Firing rate (all time points)")
plt.savefig(HERE.parent / "figures"/ f"rates {suite2p_type} {type.strip(".npy")}")


In [ ]:

movement_paths = list((HERE.parent /   "cache" ).glob("*-diffed.npy"))

movement_data = {}

for path in movement_paths:
    mouse = path.name.split("-")[0]
    assert mouse[:3] == "J02"

    movement_data[mouse] = np.load(path)

In [ ]:
plt.close('all')
plt.figure()
wt_legend_done = False
ctko_lend_done = False
for mouse in movement_data.keys():
    print(mouse)

    genotype = "WT" if int(mouse[3]) % 2 == 0 else "CTKO"
    print(genotype)
    print(genotype == "WT")
    rates = data[mouse]["data"]
    movement  = movement_data[mouse]
    movement = normalize(movement)
    # movement = moving_average(movement, 10)
    
    percent_moving = np.sum(movement > 0.1) / len(movement)

    if genotype == "WT" and not wt_legend_done:
        label = "WT"
    elif genotype == "CTKO" and not ctko_lend_done:
        label = "CTKO"
    else:
        label = None
    print(label)


    plt.plot(percent_moving, np.mean(rates), ".", color="blue" if genotype == "WT" else "orange", label=label)

    if genotype == "WT":
        wt_legend_done = True
    if genotype == "CTKO":
        ctko_lend_done = True
    
    print("\n")


plt.savefig(f"{type}")
plt.xlabel("Percent moving")
plt.ylabel("Mean firing rate")
plt.legend()

NameError: name 'movement_data' is not defined